#### Objetivo
Nosso objetivo nesta análise é descobrir casos de pessoas que doaram para campanhas a deputados federais em 2018 e estão sendo beneficiadas por recursos oriundos da cota parlamentar. Ressalto que não há vedação legal para esse tipo de situação, embora entende que pode ser questionado eventual conflito de interesse.
Vamos utilizar os dados abertos da Câmara (cotas) e do TSE (receitas de campanha).

#### Pacotes utilizados

In [29]:
import pandas as pd ### Pacote para análise e transformação de tabelas de dados

### 1. Carregando e preparando dados da Câmara e do TSE

#### -> Dados de reembolso de cota parlamentar

In [7]:
df_cota = pd.read_excel('../_dados/cota/Ano-2020.xlsx', encoding = 'latin1')
df_cota = pd.concat([df_cota, pd.read_excel('../_dados/cota/Ano-2019.xlsx', encoding = 'latin1')])

In [30]:
df_cota.head(3)

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LIDERANÇA DO DEMOCRATAS,NaN,NaN,NaN,2019,NaN,NaN,56,4,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",...,2020,0,NaN,NaN,1693866,NaN,NaN,2713,7050172,http://camara.leg.br/cota-parlamentar/nota-fis...
1,LIDERANÇA DO DEMOCRATAS,NaN,NaN,NaN,2019,NaN,NaN,56,4,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",...,2020,0,NaN,NaN,1698547,NaN,NaN,2713,7058437,http://camara.leg.br/cota-parlamentar/nota-fis...
2,LIDERANÇA DO DEMOCRATAS,NaN,NaN,NaN,2019,NaN,NaN,56,4,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",...,2020,0,NaN,NaN,1709149,NaN,NaN,2713,7078508,http://camara.leg.br/cota-parlamentar/nota-fis...


In [9]:
df_cota.columns

Index(['txNomeParlamentar', 'cpf', 'ideCadastro', 'nuCarteiraParlamentar',
       'nuLegislatura', 'sgUF', 'sgPartido', 'codLegislatura', 'numSubCota',
       'txtDescricao', 'numEspecificacaoSubCota', 'txtDescricaoEspecificacao',
       'txtFornecedor', 'txtCNPJCPF', 'txtNumero', 'indTipoDocumento',
       'datEmissao', 'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes',
       'numAno', 'numParcela', 'txtPassageiro', 'txtTrecho', 'numLote',
       'numRessarcimento', 'vlrRestituicao', 'nuDeputadoId', 'ideDocumento',
       'urlDocumento'],
      dtype='object')

In [31]:
# Criando tabela só com dados da legislatura atual
df_cota_leg_2019 = df_cota[df_cota['nuLegislatura'] == 2019].reset_index(drop = True)

In [33]:
# Ajustando cnpj do doador
def corrige_cpf_cnpj(x):
    if pd.isna(x) == True:
        y = 'nan'
    else:
        cnpj_str = x.replace('.', '').replace('/', '').replace('-', '').replace(' ', '')
        y = '0'*(14 - len(cnpj_str)) + cnpj_str
    return y

df_cota_leg_2019['txtCNPJCPF'] = df_cota_leg_2019['txtCNPJCPF'].map(corrige_cpf_cnpj)

In [34]:
# Criando dimensão com cpf do deputado
tab_cpf = df_cota_leg_2019[['txNomeParlamentar', 'cpf']].drop_duplicates().reset_index(drop = 1)

In [35]:
tab_cpf = tab_cpf[pd.isna(tab_cpf['cpf']) == False]

In [36]:
# Ajustando cpf do parlamentar na tabela auxiliar

tab_cpf['cpf_ajustado'] = tab_cpf['cpf'].apply(lambda x: '0'*(11 - len(str(int(x)))) + str(int(x)))

In [37]:
# Trazendo cpf_ajustado para tabela de cota

df_cota_leg_2019 = df_cota_leg_2019.merge(tab_cpf[['txNomeParlamentar', 'cpf_ajustado']], on = 'txNomeParlamentar', how = 'inner')


#### -> Dados do TSE: receitas

In [52]:
df_receitas_tse = pd.read_csv('../_dados/cota/receitas_candidatos_2018_BRASIL.csv', sep = ';', encoding = 'latin1')

In [53]:
df_receitas_tse.head(3)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,ST_TURNO,TP_PRESTACAO_CONTAS,...,DS_CARGO_CANDIDATO_DOADOR,NR_PARTIDO_DOADOR,SG_PARTIDO_DOADOR,NM_PARTIDO_DOADOR,NR_RECIBO_DOACAO,NR_DOCUMENTO_DOACAO,SQ_RECEITA,DT_RECEITA,DS_RECEITA,VR_RECEITA
0,29/10/2020,15:23:43,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,Vice-governador,43,PV,Partido Verde,#NULO#,702437,12216154,05/11/2018,#NULO#,"29,00"
1,29/10/2020,15:23:43,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,Vice-governador,43,PV,Partido Verde,#NULO#,694076,12216153,29/10/2018,#NULO#,"1795,00"
2,29/10/2020,15:23:43,2018,2,Ordinária,297,Eleição Geral Federal 2018,07/10/2018,1,Final,...,Governador,43,PV,Partido Verde,000430300000MS000005E,#NULO#,12317815,17/08/2018,CESSÃO DE USO DO ESPAÇO FÍSICO DO SEU IMÓVEL P...,"1500,00"


In [54]:
df_receitas_tse.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'ST_TURNO',
       'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS',
       'SG_UF', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO',
       'DS_CARGO', 'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO',
       'NR_CPF_CANDIDATO', 'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'SG_PARTIDO',
       'NM_PARTIDO', 'CD_FONTE_RECEITA', 'DS_FONTE_RECEITA',
       'CD_ORIGEM_RECEITA', 'DS_ORIGEM_RECEITA', 'CD_NATUREZA_RECEITA',
       'DS_NATUREZA_RECEITA', 'CD_ESPECIE_RECEITA', 'DS_ESPECIE_RECEITA',
       'CD_CNAE_DOADOR', 'DS_CNAE_DOADOR', 'NR_CPF_CNPJ_DOADOR', 'NM_DOADOR',
       'NM_DOADOR_RFB', 'CD_ESFERA_PARTIDARIA_DOADOR',
       'DS_ESFERA_PARTIDARIA_DOADOR', 'SG_UF_DOADOR', 'CD_MUNICIPIO_DOADOR',
       'NM_MUNICIPIO_DOADOR', 'SQ_CANDIDATO_DOADOR', 'NR_CANDIDATO_DOADOR',
       'CD_CARGO_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR

In [55]:
# Filtrando apenas recursos de pessoas físicas

df_receitas_tse = df_receitas_tse[df_receitas_tse['DS_ORIGEM_RECEITA'] == 'Recursos de pessoas físicas']

In [56]:
# Ajustando cpf do doador
df_receitas_tse['NR_CPF_CNPJ_DOADOR'] = df_receitas_tse['NR_CPF_CNPJ_DOADOR'].apply(lambda x: '0'*(14 - len(str(x))) + str(x))


In [57]:
# Ajustando valor
df_receitas_tse['VR_RECEITA'] = df_receitas_tse['VR_RECEITA'].apply(lambda x: float(x.replace(',', '.')))


In [58]:
# Ajustando CPF do candidato
df_receitas_tse['NR_CPF_CANDIDATO'] = df_receitas_tse['NR_CPF_CANDIDATO'].apply(lambda x: '0'*(11 - len(str(x))) + str(x))

### 2. Criando tabelas agregadas, candidato x doador e deputado x empresa

In [59]:
df_cota_agregado = df_cota_leg_2019[['cpf_ajustado', 'txNomeParlamentar', 'txtCNPJCPF', 'txtFornecedor', 'vlrLiquido']].\
                   groupby(['cpf_ajustado', 'txNomeParlamentar', 'txtCNPJCPF', 'txtFornecedor']).sum().reset_index()


In [60]:
df_cota_agregado.head(3)

,cpf_ajustado,txNomeParlamentar,txtCNPJCPF,txtFornecedor,vlrLiquido
0,00001608657,Stefano Aguiar,00000000000006,RAMAL,1641.69
1,00001608657,Stefano Aguiar,00000000000007,CORREIOS - CAIXA DE ENCOMENDAS BÁSICA/CONVENCI...,29.05
2,00001608657,Stefano Aguiar,00000000000007,CORREIOS - CARTA COMERCIAL,654.00


In [61]:
df_receitas_tse_agregado = df_receitas_tse[['NR_CPF_CNPJ_DOADOR', 'NM_DOADOR_RFB', 'NR_CPF_CANDIDATO', 'NM_CANDIDATO', 'VR_RECEITA']].\
                           groupby(['NR_CPF_CNPJ_DOADOR', 'NM_DOADOR_RFB', 'NR_CPF_CANDIDATO', 'NM_CANDIDATO']).sum().reset_index()


In [62]:
df_receitas_tse_agregado.head(3)

,NR_CPF_CNPJ_DOADOR,NM_DOADOR_RFB,NR_CPF_CANDIDATO,NM_CANDIDATO,VR_RECEITA
0,00000000000000,#NULO#,22333843349,ANA CLAUDIA CALLOU MATOS,80.0
1,00000000000001,#NULO#,85989746768,PATRICIA MESSINA MEDEIROS,100.0
2,00000000000002,#NULO#,85989746768,PATRICIA MESSINA MEDEIROS,50.0


### 3. Testando hipótese (empresa doadora é beneficiária de recursos da cota)

In [63]:
df_ocorrencias = df_cota_agregado.merge(df_receitas_tse_agregado[['NR_CPF_CNPJ_DOADOR', 'NR_CPF_CANDIDATO', 'VR_RECEITA']],\
                                        how = 'inner', left_on = ['cpf_ajustado', 'txtCNPJCPF'], right_on = ['NR_CPF_CANDIDATO', 'NR_CPF_CNPJ_DOADOR'])


In [64]:
df_ocorrencias

,cpf_ajustado,txNomeParlamentar,txtCNPJCPF,txtFornecedor,vlrLiquido,NR_CPF_CNPJ_DOADOR,NR_CPF_CANDIDATO,VR_RECEITA
0,01739396782,Silvia Cristina,00019103743268,MARINETE CALIMAN FRANCISCO,143000.00,00019103743268,01739396782,500.0
1,02112481770,David Soares,00022002077860,LUCAS JOSE VIEIRA,1023.45,00022002077860,02112481770,7120.0


Os resultdos acima foram os únicos 2 registros identificados em que o CPF que doou para a campanha do deputado vem recebendo recursos do mesmo deputado via cota parlamentar.